<a href="https://colab.research.google.com/github/minonine/webscraping_indeed/blob/main/Mini_projet_%C3%A0_faire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Projet : APACHE SPARK

In [3]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"



In [4]:
# install findspark using pip
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 65.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f77bc79f011b1e7bd1108e8d4c5f0fdcfde358a2118fd38c9d88bfabb9829d36
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


### 1 - Creation de session

In [5]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [7]:
# import SparkContext
# sc = SparkContext("local", "firstSpark")
spark = SparkSession \
        .builder \
        .appName("firstSpark") \
        .getOrCreate()
SparkSession.getActiveSession()

### 2 - Ingestion du CSV Donors

In [8]:
df_donations = spark.read.format('csv').options(header='true').load('Donations.csv')
df_donors = spark.read.format('csv').options(header='true').load('Donors.csv')
df_donors2 = spark.read.format('csv').options(header='true').load('Donors.csv')

### 3 - Affichage
Afficher les 20 premières lignes de dataset Donors (Utilisez la fonction show())

In [9]:
df_donors.show()

+--------------------+------------+--------------+----------------+---------+
|            Donor_ID|  Donor_City|   Donor_State|Donor_Is_Teacher|Donor_Zip|
+--------------------+------------+--------------+----------------+---------+
|00000ce845c00cbf0...|    Evanston|      Illinois|              No|      602|
|00002783bc5d10851...|  Appomattox|         other|              No|      245|
|00002d44003ed46b0...|      Winton|    California|             Yes|      953|
|00002eb25d60a09c3...|Indianapolis|       Indiana|              No|      462|
|0000300773fe015f8...|    Paterson|    New Jersey|              No|      075|
|00004c31ce07c2214...|        null|         other|              No|     null|
|00004e32a448b4832...|    Stamford|   Connecticut|              No|      069|
|00004fa20a986e60a...|   Green Bay|     Wisconsin|              No|      543|
|00005454366b6b914...|      Argyle|      New York|              No|      128|
|0000584b8cdaeaa6b...|  Valparaiso|       Indiana|              

Conversion en dataframe pandas (utilisez la fonction "toPandas()")

In [10]:
df_donors.toPandas()

,Donor_ID,Donor_City,Donor_State,Donor_Is_Teacher,Donor_Zip
0,00000ce845c00cbf0686c992fc369df4,Evanston,Illinois,No,602
1,00002783bc5d108510f3f9666c8b1edd,Appomattox,other,No,245
2,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953
3,00002eb25d60a09c318efbd0797bffb5,Indianapolis,Indiana,No,462
4,0000300773fe015f870914b42528541b,Paterson,New Jersey,No,075
...,...,...,...,...,...
1048570,7e60d56f25337e420790991868376fdd,Salt Lake City,Utah,No,841
1048571,7e60e11156fdfa691825a2b10abc17cf,Dayton,Ohio,No,454
1048572,7e60f789db33462daf34ab795c4d70fe,Phoenix,Arizona,No,850
1048573,7e610c8ed745f8307edf0c4257e1a455,Greer,South Carolina,No,296


Trouver le nombre nul dans chaque colonne

In [12]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df_donors.select([count(when(isnan(column), column)).alias(column) for column in df_donors.columns]).show()
[df_donors.filter(col(column).isNull()).count() for column in df_donors.columns]

+--------+----------+-----------+----------------+---------+
|Donor_ID|Donor_City|Donor_State|Donor_Is_Teacher|Donor_Zip|
+--------+----------+-----------+----------------+---------+
|       0|         0|          0|               0|        0|
+--------+----------+-----------+----------------+---------+



[0, 105086, 0, 0, 88694]

Imprimer le schéma de dataset (pour imprimer le schéma, on utilise la fonction "printSchema")

In [13]:
df_donors.printSchema()

root
 |-- Donor_ID: string (nullable = true)
 |-- Donor_City: string (nullable = true)
 |-- Donor_State: string (nullable = true)
 |-- Donor_Is_Teacher: string (nullable = true)
 |-- Donor_Zip: string (nullable = true)



### 4 - Filtrage

#### Laissez que les enregistrement dont Donor City commence par A

Vous pouvez utiliser la fonction "filter" 
 
Exemple :  "My_data.filter(My_data.name_colonne.like("A%"))"

Like("A%") : le caractère "%" est un caractère joker qui remplace tous les autres caractères. Ainsi, ce modèle permet de rechercher toutes les chaines de caractère qui commence par un "A".

In [14]:
import pandas as pd
start_with_A=pd.DataFrame(df_donors.filter(df_donors['Donor_City'].startswith('A')).collect())
df=df_donors.filter(df_donors.Donor_City.like("A%"))

#### Affichez les résultats

In [15]:
type(df_donors)
df.show()

+--------------------+-------------+-------------+----------------+---------+
|            Donor_ID|   Donor_City|  Donor_State|Donor_Is_Teacher|Donor_Zip|
+--------------------+-------------+-------------+----------------+---------+
|00002783bc5d10851...|   Appomattox|        other|              No|      245|
|00005454366b6b914...|       Argyle|     New York|              No|      128|
|0001ef9f64a7e1038...|         Ames|         Iowa|              No|      500|
|00024e86676fc2c3b...|    Ann Arbor|     Michigan|              No|      481|
|0002a45d0b45a78e9...|    Ann Arbor|     Michigan|              No|      481|
|0002cb56c84b1cba7...|         Avon|  Connecticut|              No|      060|
|00050297e37eb7632...|       Austin|        Texas|              No|      787|
|0005327bfe18229b9...|        Acton|   California|              No|      935|
|00054f4b278af0c8d...|American Fork|         Utah|              No|      840|
|00055ed4f4745e71d...|  Albuquerque|   New Mexico|              

## 5 - Transformation

#### Construisez une nouvelle colonne `Address` en faisant une concaténation `Donor_City, Donor_State, Donor_Zip`

In [16]:
from pyspark.sql.functions import concat_ws

Vous pouvez utiliser la fonction "withColumn" et "concat_ws" 

In [17]:
from pyspark.sql import functions as sf
df_donors = df_donors.withColumn('Address ', 
                    sf.concat(sf.col('Donor_City'),sf.lit('_'), sf.col('Donor_State'),sf.lit('_'), sf.col('Donor_Zip')))



Afficher les résultats

In [18]:
df_donors.show()

+--------------------+------------+--------------+----------------+---------+--------------------+
|            Donor_ID|  Donor_City|   Donor_State|Donor_Is_Teacher|Donor_Zip|            Address |
+--------------------+------------+--------------+----------------+---------+--------------------+
|00000ce845c00cbf0...|    Evanston|      Illinois|              No|      602|Evanston_Illinois...|
|00002783bc5d10851...|  Appomattox|         other|              No|      245|Appomattox_other_245|
|00002d44003ed46b0...|      Winton|    California|             Yes|      953|Winton_California...|
|00002eb25d60a09c3...|Indianapolis|       Indiana|              No|      462|Indianapolis_Indi...|
|0000300773fe015f8...|    Paterson|    New Jersey|              No|      075|Paterson_New Jers...|
|00004c31ce07c2214...|        null|         other|              No|     null|                null|
|00004e32a448b4832...|    Stamford|   Connecticut|              No|      069|Stamford_Connecti...|
|00004fa20

## 6 - Moteur SQL

#### Persister le dataset de départ comme une Temporary View
Vous pouvez utliser la fcontion createOrReplaceTempView

In [19]:
df_donors.createOrReplaceTempView("donors")
result = spark.sql(""" select * from donors """)
result.show()

+--------------------+------------+--------------+----------------+---------+--------------------+
|            Donor_ID|  Donor_City|   Donor_State|Donor_Is_Teacher|Donor_Zip|            Address |
+--------------------+------------+--------------+----------------+---------+--------------------+
|00000ce845c00cbf0...|    Evanston|      Illinois|              No|      602|Evanston_Illinois...|
|00002783bc5d10851...|  Appomattox|         other|              No|      245|Appomattox_other_245|
|00002d44003ed46b0...|      Winton|    California|             Yes|      953|Winton_California...|
|00002eb25d60a09c3...|Indianapolis|       Indiana|              No|      462|Indianapolis_Indi...|
|0000300773fe015f8...|    Paterson|    New Jersey|              No|      075|Paterson_New Jers...|
|00004c31ce07c2214...|        null|         other|              No|     null|                null|
|00004e32a448b4832...|    Stamford|   Connecticut|              No|      069|Stamford_Connecti...|
|00004fa20

#### Comptez le nombre de professeurs ayant participé à la donation

Vous pouvez utiliser la fonction count() et le langage SQL

In [20]:
result = spark.sql(""" select count(*) from donors Where Donor_Is_Teacher='Yes' """)
result.show()

+--------+
|count(1)|
+--------+
|  104650|
+--------+



utiliser juste 10% du dataset c'est très grand complet pour des jointures pour votre petite machine... avec la method `sample`

In [22]:
df_e = df_donors.sample(fraction=0.1, seed=3)
df_e.createOrReplaceTempView("donor")
result = spark.sql(""" select Donor_ID from donor """)
result.show()

+--------------------+
|            Donor_ID|
+--------------------+
|0000c4be79d9f9248...|
|0001415ef50c0f45d...|
|000150db429aff026...|
|00015a61076990002...|
|00016d53aafa42950...|
|000177bef7ed7b7d1...|
|0002128c613edd04b...|
|0002348747db8f7fc...|
|0002965240b13a26f...|
|0003982f3e5d7e11e...|
|0004022cc26b79c37...|
|0004265c44e425d71...|
|000446688c194fc58...|
|00049338d2a420cd9...|
|0004c11d84e7e1fc8...|
|0004f42bf17585076...|
|00050511b93260bed...|
|0005217360eaad75d...|
|0005f200ba504e18b...|
|00061145ab79728ae...|
+--------------------+
only showing top 20 rows



#### Afficher que les id des donateurs qui habite à California

Vous pouvez utiliser le langage SQL qu'on vu dans le TP 5 suivant select col_x from donors where col_y ="California

In [23]:
result = spark.sql(""" select Donor_City from donor where Donor_City='California' """)
result.show()

+----------+
|Donor_City|
+----------+
|California|
|California|
|California|
|California|
|California|
|California|
+----------+



#### Ingestion des données et publication en temporary view du fichier Donations.CSV

In [24]:
df_donations.createOrReplaceTempView("donations")

In [25]:
df_donations.show()

+--------------------+--------------------+--------------------+-----------------------------------+---------------+-------------------+----------------------+
|          Project ID|         Donation_ID|            Donor_ID|Donation_Included_Optional_Donation|Donation_Amount|Donor_Cart Sequence|Donation_Received_Date|
+--------------------+--------------------+--------------------+-----------------------------------+---------------+-------------------+----------------------+
|000009891526c0ade...|68872912085866622...|1f4b5b6e68445c6c4...|                                 No|         178.37|                 11|   2016-08-23 13:15:57|
|000009891526c0ade...|dcf1071da3aa3561f...|4aaab6d244bf35996...|                                Yes|           25.0|                  2|   2016-06-06 20:05:23|
|000009891526c0ade...|18a234b9d1e538c43...|0b0765dc9c759adc4...|                                Yes|           20.0|                  3|   2016-06-06 14:08:46|
|000009891526c0ade...|38d2744bf9138b0b5.

#### Afficher le DF

In [26]:
df_donations.count()
df_donations.show()

+--------------------+--------------------+--------------------+-----------------------------------+---------------+-------------------+----------------------+
|          Project ID|         Donation_ID|            Donor_ID|Donation_Included_Optional_Donation|Donation_Amount|Donor_Cart Sequence|Donation_Received_Date|
+--------------------+--------------------+--------------------+-----------------------------------+---------------+-------------------+----------------------+
|000009891526c0ade...|68872912085866622...|1f4b5b6e68445c6c4...|                                 No|         178.37|                 11|   2016-08-23 13:15:57|
|000009891526c0ade...|dcf1071da3aa3561f...|4aaab6d244bf35996...|                                Yes|           25.0|                  2|   2016-06-06 20:05:23|
|000009891526c0ade...|18a234b9d1e538c43...|0b0765dc9c759adc4...|                                Yes|           20.0|                  3|   2016-06-06 14:08:46|
|000009891526c0ade...|38d2744bf9138b0b5.

#### Calculer le montant minimum, le montant maximum, le montant moyen en arrondissant à l'unité après la virgule de la colonne Donation_Amount
pour l'ensemble Donations 

Utiliser les alias "maxMontant", "minMontant", "avgMontant". et la colonne "Donation_Amount"

Pour rappel en SQL, un alias ressemble à ça : "as maxMontant".

In [27]:
result = spark.sql(""" select Max(Donation_Amount) as maxMontant , MIN(Donation_Amount) as minMontant , AVG(Donation_Amount) as avgMontant from donations  """)
result.show()

+----------+----------+-----------------+
|maxMontant|minMontant|       avgMontant|
+----------+----------+-----------------+
|   9999.91|      0.01|60.72797862813836|
+----------+----------+-----------------+



utiliser juste 10% du dataset c'est très grand complet pour des jointures... avec la method `sample`

In [31]:
donations_e = df_donations.sample(fraction=0.1, seed=3)
donations_e.createOrReplaceTempView("donations_e")

#### Faites une jointure Entre le data set des donneurs Donors, et le dataset des Donations Donations

Indication : utilisez "inner join" de langage spark.sql

In [37]:
result = spark.sql("SELECT * FROM Donors INNER JOIN Donations ON Donors.Donor_id == Donations.Donor_id")
result.show()

+--------------------+----------------+--------------+----------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------------------------+---------------+-------------------+----------------------+
|            Donor_ID|      Donor_City|   Donor_State|Donor_Is_Teacher|Donor_Zip|            Address |          Project ID|         Donation_ID|            Donor_ID|Donation_Included_Optional_Donation|Donation_Amount|Donor_Cart Sequence|Donation_Received_Date|
+--------------------+----------------+--------------+----------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------------------------+---------------+-------------------+----------------------+
|000eb10b7e8fb44e9...|      Manchester|   Connecticut|              No|      060|Manchester_Connec...|123050acb1743c4b8...|84928abc68304ea56...|000eb10b7e8fb44e9...|                                Yes|           25.0|

#### Calculez la somme de l'argent donnée par Les Professeurs (Donor Is Teacher=Yes) et les non professeurs utilisant seulement SQL

Indication : ('select sum(dt.col4) as amountProf from donations dt inner join donors dr on dt.col2 = dr.col0 and dr.col3 = "Yes" ')

In [38]:
result = spark.sql(""" select Sum(Donation_Amount) from donations_e INNER JOIN donor where donations_e.Donor_ID=donor.Donor_ID  """)
result.show()

+------------------------------------+
|sum(CAST(Donation_Amount AS DOUBLE))|
+------------------------------------+
|                  319888.66000000003|
+------------------------------------+

